# Otimização do Mix de Culturas — Versão com Diversificação

Adiciona restrição mínima de plantio por cultura (5% da área não compactada).

In [1]:
import pandas as pd
import numpy as np
import pulp
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, value, LpStatus


In [2]:
def generate_mock_data(n_dados=1500, seed=42):
    np.random.seed(seed)
    dados = {
        'id_talhao': range(1, n_dados + 1),
        'cultura': np.random.choice(['Soja_Resistente', 'Soja_Produtiva', 'Milho_Safrinha'], n_dados),
    }
    df = pd.DataFrame(dados)

    culturas_params = {
        'Soja_Resistente': {'prod': (3.5, 0.5), 'custo': (1800, 150), 'agua': (450, 50), 'k': (80, 10), 'p': (70, 8), 'horas': (10, 1.5)},
        'Soja_Produtiva': {'prod': (4.8, 0.6), 'custo': (2500, 200), 'agua': (600, 60), 'k': (100, 12), 'p': (90, 10), 'horas': (12, 1.8)},
        'Milho_Safrinha': {'prod': (5.5, 0.7), 'custo': (2800, 250), 'agua': (700, 70), 'k': (120, 15), 'p': (100, 12), 'horas': (15, 2.0)}
    }

    def generate_data(params, size):
        return {
            'produtividade_ton_ha': np.random.normal(loc=params['prod'][0], scale=params['prod'][1], size=size),
            'custo_ha': np.random.normal(loc=params['custo'][0], scale=params['custo'][1], size=size),
            'uso_agua_m3_ha': np.random.normal(loc=params['agua'][0], scale=params['agua'][1], size=size),
            'demanda_k_kg_ha': np.random.normal(loc=params['k'][0], scale=params['k'][1], size=size),
            'demanda_p_kg_ha': np.random.normal(loc=params['p'][0], scale=params['p'][1], size=size),
            'horas_maquina_ha': np.random.normal(loc=params['horas'][0], scale=params['horas'][1], size=size),
        }

    for cultura, params in culturas_params.items():
        mask = df['cultura'] == cultura
        for col, values in generate_data(params, df[mask].shape[0]).items():
            df.loc[mask, col] = values

    return df


In [3]:
def setup_resources(area_total=500, orcamento=1_100_000, agua=250_000, potassio=45000, fosforo=42000, area_nao_compactada=400, horas_maquina=6000, capacidade_silo=2500):
    return {
        'AREA_TOTAL_DISPONIVEL_HA': area_total,
        'ORCAMENTO_TOTAL_DISPONIVEL': orcamento,
        'AGUA_TOTAL_DISPONIVEL_M3': agua,
        'POTASSIO_DISPONIVEL_KG': potassio,
        'FOSFORO_DISPONIVEL_KG': fosforo,
        'AREA_NAO_COMPACTADA_HA': area_nao_compactada,
        'HORAS_MAQUINA_DISPONIVEIS': horas_maquina,
        'CAPACIDADE_SILO_TON': capacidade_silo
    }


In [4]:
# Abrir arquivo de log
output_file = open('outputs-otimizacao_mix_culturas_v2_diversificacao.txt', 'w')

# Parâmetros e geração de dados
n_dados = 1500
seed = 42
preco_soja = 2200
preco_milho = 1300
df = generate_mock_data(n_dados, seed)
params = df.groupby('cultura').mean()
# calcular lucro por ha
params['lucro_ha'] = 0
params.loc['Soja_Resistente', 'lucro_ha'] = (params.loc['Soja_Resistente', 'produtividade_ton_ha'] * preco_soja) - params.loc['Soja_Resistente', 'custo_ha']
params.loc['Soja_Produtiva', 'lucro_ha'] = (params.loc['Soja_Produtiva', 'produtividade_ton_ha'] * preco_soja) - params.loc['Soja_Produtiva', 'custo_ha']
params.loc['Milho_Safrinha', 'lucro_ha'] = (params.loc['Milho_Safrinha', 'produtividade_ton_ha'] * preco_milho) - params.loc['Milho_Safrinha', 'custo_ha']
resources = setup_resources()


/tmp/ipykernel_44218/3588544693.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5914.589486273522' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  params.loc['Soja_Resistente', 'lucro_ha'] = (params.loc['Soja_Resistente', 'produtividade_ton_ha'] * preco_soja) - params.loc['Soja_Resistente', 'custo_ha']


In [5]:
def setup_model(params, resources):
    # Extrair parâmetros relevantes
    l_sr = params.loc['Soja_Resistente', 'lucro_ha']
    l_sp = params.loc['Soja_Produtiva', 'lucro_ha']
    l_ms = params.loc['Milho_Safrinha', 'lucro_ha']
    c_sr = params.loc['Soja_Resistente', 'custo_ha']
    c_sp = params.loc['Soja_Produtiva', 'custo_ha']
    c_ms = params.loc['Milho_Safrinha', 'custo_ha']
    a_sr = params.loc['Soja_Resistente', 'uso_agua_m3_ha']
    a_sp = params.loc['Soja_Produtiva', 'uso_agua_m3_ha']
    a_ms = params.loc['Milho_Safrinha', 'uso_agua_m3_ha']
    k_sr, p_sr = params.loc['Soja_Resistente', ['demanda_k_kg_ha', 'demanda_p_kg_ha']]
    k_sp, p_sp = params.loc['Soja_Produtiva', ['demanda_k_kg_ha', 'demanda_p_kg_ha']]
    k_ms, p_ms = params.loc['Milho_Safrinha', ['demanda_k_kg_ha', 'demanda_p_kg_ha']]
    h_sr = params.loc['Soja_Resistente', 'horas_maquina_ha']
    h_sp = params.loc['Soja_Produtiva', 'horas_maquina_ha']
    h_ms = params.loc['Milho_Safrinha', 'horas_maquina_ha']
    prod_sr = params.loc['Soja_Resistente', 'produtividade_ton_ha']
    prod_sp = params.loc['Soja_Produtiva', 'produtividade_ton_ha']
    prod_ms = params.loc['Milho_Safrinha', 'produtividade_ton_ha']

    modelo = LpProblem(name='Otimizacao_Mix_Culturas', sense=LpMaximize)

    x_sr = LpVariable('Hectares_Soja_Resistente', lowBound=0, cat='Continuous')
    x_sp = LpVariable('Hectares_Soja_Produtiva', lowBound=0, cat='Continuous')
    x_ms = LpVariable('Hectares_Milho_Safrinha', lowBound=0, cat='Continuous')

    # Diversificação: mínimo 15% da área não compactada por cultura
    percentual_minimo_por_cultura = 0.15
    area_minima_ha = resources['AREA_NAO_COMPACTADA_HA'] * percentual_minimo_por_cultura
    try:
        output_file.write(f'Adicionando restrição de plantio mínimo: {area_minima_ha:.2f} ha por cultura.\n')
    except Exception:
        pass

    modelo += (x_sr >= area_minima_ha, 'Minimo_Soja_Resistente')
    modelo += (x_sp >= area_minima_ha, 'Minimo_Soja_Produtiva')
    modelo += (x_ms >= area_minima_ha, 'Minimo_Milho_Safrinha')

    modelo += lpSum([l_sr * x_sr, l_sp * x_sp, l_ms * x_ms]), 'Lucro_Total'

    modelo += (x_sr + x_sp + x_ms <= resources['AREA_TOTAL_DISPONIVEL_HA'], 'Restricao_Area_Total')
    modelo += (x_sr + x_sp + x_ms <= resources['AREA_NAO_COMPACTADA_HA'], 'Restricao_Area_Nao_Compactada')
    modelo += (c_sr * x_sr + c_sp * x_sp + c_ms * x_ms <= resources['ORCAMENTO_TOTAL_DISPONIVEL'], 'Restricao_Orcamento')
    modelo += (a_sr * x_sr + a_sp * x_sp + a_ms * x_ms <= resources['AGUA_TOTAL_DISPONIVEL_M3'], 'Restricao_Agua')
    modelo += (k_sr * x_sr + k_sp * x_sp + k_ms * x_ms <= resources['POTASSIO_DISPONIVEL_KG'], 'Restricao_Potassio')
    modelo += (p_sr * x_sr + p_sp * x_sp + p_ms * x_ms <= resources['FOSFORO_DISPONIVEL_KG'], 'Restricao_Fosforo')
    modelo += (h_sr * x_sr + h_sp * x_sp + h_ms * x_ms <= resources['HORAS_MAQUINA_DISPONIVEIS'], 'Restricao_Horas_Maquina')
    modelo += (prod_sr * x_sr + prod_sp * x_sp + prod_ms * x_ms <= resources['CAPACIDADE_SILO_TON'], 'Restricao_Armazenagem')

    return modelo, (x_sr, x_sp, x_ms)


In [6]:
def solve_and_print(modelo, variables):
    x_sr, x_sp, x_ms = variables
    modelo.solve()
    status = LpStatus[modelo.status]
    print(f'Status da Solução: {status}')
    output_file.write(f'Status da Solução: {status}\n')
    if status == 'Optimal':
        print('Plano de Plantio Ótimo:')
        output_file.write('Plano de Plantio Ótimo:\n')
        print(f'  - Plantar {x_sr.varValue:.2f} ha de Soja Resistente.')
        output_file.write(f'  - Plantar {x_sr.varValue:.2f} ha de Soja Resistente.\n')
        print(f'  - Plantar {x_sp.varValue:.2f} ha de Soja Produtiva.')
        output_file.write(f'  - Plantar {x_sp.varValue:.2f} ha de Soja Produtiva.\n')
        print(f'  - Plantar {x_ms.varValue:.2f} ha de Milho Safrinha.')
        output_file.write(f'  - Plantar {x_ms.varValue:.2f} ha de Milho Safrinha.\n')
        print(f'\nLucro Máximo Esperado: R$ {value(modelo.objective):,.2f}')
        output_file.write(f'\nLucro Máximo Esperado: R$ {value(modelo.objective):,.2f}\n')
    else:
        print('Não foi possível encontrar uma solução ótima. Verifique as restrições.')
        output_file.write('Não foi possível encontrar uma solução ótima. Verifique as restrições.\n')
    return status

In [7]:
def sensitivity_analysis(modelo, status):
    if status == 'Optimal':
        print('\nPreço Sombra (Shadow Price) dos Recursos:')
        output_file.write('\nPreço Sombra (Shadow Price) dos Recursos:\n')
        for nome, restricao in modelo.constraints.items():
            preco_sombra = getattr(restricao, 'pi', None)
            if preco_sombra is None:
                text = f'  - Recurso {nome}: Preço Sombra não disponível'
                print(text)
                output_file.write(text + '\n')
            else:
                text = f'  - Recurso {nome}: Preço Sombra = R$ {preco_sombra:.2f}'
                print(text)
                output_file.write(text + '\n')
            if preco_sombra is not None and preco_sombra > 0:
                gargalo = '    -> GARGALO! Cada unidade a mais aumentaria o lucro em R$ {:.2f}.'.format(preco_sombra)
                print(gargalo)
                output_file.write(gargalo + '\n')
    else:
        print('Análise de sensibilidade não aplicável.')
        output_file.write('Análise de sensibilidade não aplicável.\n')

In [8]:
# Configurar e resolver o modelo
modelo, variables = setup_model(params, resources)
status = solve_and_print(modelo, variables)
sensitivity_analysis(modelo, status)
output_file.close()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/vinicius/Downloads/estudo/po/PL/venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/f02fb940d68b4befb75aa5a44c15e91a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/f02fb940d68b4befb75aa5a44c15e91a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 47 RHS
At line 59 BOUNDS
At line 60 ENDATA
Problem MODEL has 11 rows, 3 columns and 27 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-4) rows, 3 (0) columns and 21 (-6) elements
0  Obj 1100770.2 Dual inf 18346.17 (3)
1  Obj 2866818.2
Optimal - objective value 2866818.2
After Postsolve, objective 2866818.2, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 2866818.172 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CP